In [ ]:
import pandas as pd
import gzip,json,os,openai

openai.api_key  = os.getenv('OPENAI_API_KEY')


In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

#https://nijianmo.github.io/amazon/index.html

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [58]:
#getting text reviews
df = getDF('Appliances_5.json.gz')
text_random = df.query('reviewText.str.len()>50 & reviewText.str.len()<200').sample(n=10)


In [60]:
# text_5 = text_review.head(5)
for index,row in text_random.iterrows():
    text = row['reviewText']
    prompt = f"""
        Use the text delimited by tripe backticks for the following exercise
        Extract the sentiment from text, it can only be positive, negative or neutral
        If the text suggests something, extract it and summarize the text in 10 words

        Give the output in json format with the following keys
        text in full, sentiment, sentiment keywords, summmary

        ```{text}```
    """
    response = get_completion(prompt)
    print(response)




{
  "text": "I use these regular on my jobs I love the pricing too",
  "sentiment": "positive",
  "sentiment keywords": ["love", "pricing"],
  "summary": "Regular use, love the pricing on my jobs"
}
{
  "text": "Worked great!! I just wonder how long they last. Guess I'll find out. This product is very easy to install!!",
  "sentiment": "positive",
  "sentiment keywords": ["great", "easy"],
  "summary": "Product worked great, easy to install, curious about durability."
}
{
  "text": "This muti pack makes sure I always have the part to finish the job. I can be in and out with a check in hand.",
  "sentiment": "positive",
  "sentiment keywords": ["always", "finish", "job"],
  "summary": "Muti pack ensures availability of parts for quick job completion."
}
{
  "text": "Worked great. Tastes the same as the more expensive brand filters",
  "sentiment": "positive",
  "sentiment keywords": ["great", "same"],
  "summary": "Affordable filter that performs as well as expensive brands"
}
{
  "text